In [2]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder


llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    memory_key="chat_history",
    return_messages=True
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI talking to a human"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{question}")
])


def load_memory(_):
    return memory.load_memory_variables({})["chat_history"]


chain = RunnablePassthrough.assign(chat_history=load_memory) | prompt | llm


def invoke_chain(question):
    result = chain.invoke({
        "question": question
    })
    memory.save_context({"input": question}, {"output": result.content})
    print(result)
